In [1]:
import sys
#!{sys.executable} -m pip install librosa
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install scipy
#!{sys.executable} -m pip install keras
#!{sys.executable} -m pip install tensorflow
#!{sys.executable} -m pip install scikit-learn
import numpy as np, keras, librosa, tensorflow, scipy

The directory '/home/varshini/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/varshini/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/varshini/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/varshini/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The di

Using TensorFlow backend.


In [2]:
import os

In [3]:
def wav2mfcc(file_path, max_pad_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)
    if (max_pad_len>mfcc.shape[1]):
        pad_width = max_pad_len-mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0,0),(0,pad_width)),mode='constant')
    else:
        mfcc = mfcc[:,:max_pad_len]
    return mfcc

In [4]:
import numpy as np
DATA_PATH = "./data/"
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, keras.utils.to_categorical(label_indices)

In [5]:
def save_data_to_array(path=DATA_PATH, max_pad_len=11):
    labels,_,_= get_labels(path)
    
    for label in labels:
        mfcc_vectors = []
        
        wavfiles = [path+label+'/'+wavfile for wavfile in os.listdir(path+'/'+label)]
        for wavfile in wavfiles:
            mfcc = wav2mfcc(wavfile, max_pad_len=max_pad_len)
            mfcc_vectors.append(mfcc)
            
        np.save(label+'.npy',mfcc_vectors)

In [6]:
from sklearn.model_selection import train_test_split
def get_train_test(split_ratio=0.6, random_state=42):
    labels, indices,_ = get_labels(DATA_PATH)
    
    X = np.load(labels[0]+'.npy')
    y = np.zeros(X.shape[0])
    for i, label in enumerate(labels[1:]):
        x = np.load(label+'.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value=(i+1)))
        
    assert X.shape[0] == len(y)
    
    return train_test_split(X, y, test_size=(1-split_ratio),random_state=random_state, shuffle=True)

In [7]:
save_data_to_array()

In [8]:
X_train, X_test, y_train, y_test = get_train_test()
X_train = X_train.reshape(X_train.shape[0], 20, 11, 1)
X_test = X_test.reshape(X_test.shape[0], 20, 11, 1)
y_train_hot = keras.utils.to_categorical(y_train)
y_test_hot = keras.utils.to_categorical(y_test)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
model = Sequential()
model.add(Conv2D(32, kernel_size=(2,2), activation='relu',input_shape=(20,11,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(4,activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])
model.fit(X_train, y_train_hot,batch_size=100, epochs=200, verbose=1, validation_data=(X_test, y_test_hot))


Train on 5683 samples, validate on 3789 samples
Epoch 1/200
5683/5683 [==============================] - 3s 554us/step - loss: 6.8392 - acc: 0.3560 - val_loss: 1.2538 - val_acc: 0.4423
Epoch 2/200
5683/5683 [==============================] - 1s 253us/step - loss: 1.3042 - acc: 0.4353 - val_loss: 1.0603 - val_acc: 0.5442
Epoch 3/200
5683/5683 [==============================] - 1s 201us/step - loss: 1.1320 - acc: 0.5047 - val_loss: 0.8793 - val_acc: 0.6297
Epoch 4/200
5683/5683 [==============================] - 1s 204us/step - loss: 0.9978 - acc: 0.5640 - val_loss: 0.8421 - val_acc: 0.6466
Epoch 5/200
5683/5683 [==============================] - 1s 223us/step - loss: 0.9203 - acc: 0.6029 - val_loss: 0.6643 - val_acc: 0.7464
Epoch 6/200
5683/5683 [==============================] - 1s 215us/step - loss: 0.8271 - acc: 0.6484 - val_loss: 0.6956 - val_acc: 0.7305
Epoch 7/200
5683/5683 [==============================] - 1s 200us/step - loss: 0.8145 - acc: 0.6570 - val_loss: 0.5890 - val_acc: 

5683/5683 [==============================] - 1s 248us/step - loss: 0.2251 - acc: 0.9129 - val_loss: 0.2792 - val_acc: 0.9071
Epoch 61/200
5683/5683 [==============================] - 1s 230us/step - loss: 0.2245 - acc: 0.9136 - val_loss: 0.2741 - val_acc: 0.9031
Epoch 62/200
5683/5683 [==============================] - 1s 231us/step - loss: 0.2196 - acc: 0.9125 - val_loss: 0.2600 - val_acc: 0.9087
Epoch 63/200
5683/5683 [==============================] - 1s 234us/step - loss: 0.2195 - acc: 0.9108 - val_loss: 0.2571 - val_acc: 0.9079
Epoch 64/200
5683/5683 [==============================] - 1s 240us/step - loss: 0.2253 - acc: 0.9117 - val_loss: 0.2755 - val_acc: 0.9058
Epoch 65/200
5683/5683 [==============================] - 1s 238us/step - loss: 0.2117 - acc: 0.9198 - val_loss: 0.2673 - val_acc: 0.9058
Epoch 66/200
5683/5683 [==============================] - 1s 237us/step - loss: 0.2149 - acc: 0.9187 - val_loss: 0.2746 - val_acc: 0.9018
Epoch 67/200
5683/5683 [=======================

5683/5683 [==============================] - 1s 231us/step - loss: 0.1277 - acc: 0.9514 - val_loss: 0.2894 - val_acc: 0.9103
Epoch 120/200
5683/5683 [==============================] - 1s 232us/step - loss: 0.1391 - acc: 0.9477 - val_loss: 0.2758 - val_acc: 0.9121
Epoch 121/200
5683/5683 [==============================] - 1s 232us/step - loss: 0.1372 - acc: 0.9520 - val_loss: 0.2769 - val_acc: 0.9132
Epoch 122/200
5683/5683 [==============================] - 1s 234us/step - loss: 0.1352 - acc: 0.9490 - val_loss: 0.2772 - val_acc: 0.9161
Epoch 123/200
5683/5683 [==============================] - 1s 232us/step - loss: 0.1189 - acc: 0.9542 - val_loss: 0.2909 - val_acc: 0.9129
Epoch 124/200
5683/5683 [==============================] - 1s 235us/step - loss: 0.1344 - acc: 0.9474 - val_loss: 0.2696 - val_acc: 0.9187
Epoch 125/200
5683/5683 [==============================] - 1s 234us/step - loss: 0.1363 - acc: 0.9502 - val_loss: 0.2801 - val_acc: 0.9126
Epoch 126/200
5683/5683 [================

5683/5683 [==============================] - 1s 232us/step - loss: 0.0932 - acc: 0.9622 - val_loss: 0.3072 - val_acc: 0.9155
Epoch 179/200
5683/5683 [==============================] - 1s 230us/step - loss: 0.0937 - acc: 0.9636 - val_loss: 0.3042 - val_acc: 0.9108
Epoch 180/200
5683/5683 [==============================] - 1s 251us/step - loss: 0.0941 - acc: 0.9632 - val_loss: 0.3093 - val_acc: 0.9142
Epoch 181/200
5683/5683 [==============================] - 1s 231us/step - loss: 0.1063 - acc: 0.9592 - val_loss: 0.3023 - val_acc: 0.9148
Epoch 182/200
5683/5683 [==============================] - 1s 234us/step - loss: 0.1064 - acc: 0.9595 - val_loss: 0.3315 - val_acc: 0.9097
Epoch 183/200
5683/5683 [==============================] - 1s 248us/step - loss: 0.0924 - acc: 0.9641 - val_loss: 0.3040 - val_acc: 0.9121
Epoch 184/200
5683/5683 [==============================] - 1s 245us/step - loss: 0.0896 - acc: 0.9676 - val_loss: 0.2780 - val_acc: 0.9142
Epoch 185/200
5683/5683 [================

In [10]:
sample = wav2mfcc('./GoRecording3.wav')
sample_reshaped = sample.reshape(1,20,11,1)
print(get_labels()[0][np.argmax(model.predict(sample_reshaped))])

go
